In [1]:
%load_ext autoreload
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%reload_ext autoreload
%matplotlib inline
%autoreload 2

!pip3 install --upgrade pip
!pip3 install --upgrade torch
!pip3 install torchvision
!pip3 install torchfusion
!pip3 install tensorboardx
!pip3 install pillow
!pip3 install pydicom
!pip3 install --upgrade keras
!pip3 install --upgrade keras.utils
!pip3 install opencv-python
!pip3 install nibabel

import os
import io
import errno
import scipy
import pydicom as dicom
import scipy.misc
import numpy as np
import tqdm
import random
import nibabel as nib
import datetime
from IPython.display import clear_output

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow import nn, layers
from tensorflow.contrib import layers as clayers 

import keras
import keras.backend as K
from keras.layers.core import Dropout, Lambda, Dense, Flatten
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.merge import Add, Concatenate
from keras.layers import concatenate
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Activation, Flatten, ZeroPadding2D, UpSampling2D, MaxPooling2D, merge, Convolution2D, Cropping2D
from keras.models import Model
from keras.optimizers import Adam
from keras.engine import InputSpec
from keras.engine.topology import Layer
from keras.utils import conv_utils
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras import losses

import torch
import torch.cuda as cuda
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader, random_split
from torch.distributions import Normal

import torchvision.utils as vutils
from torchvision import transforms, utils, datasets

from tensorboardX import SummaryWriter
from IPython import display

from PIL import Image

from getpass import getpass

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import warnings
warnings.filterwarnings(action = 'once')

    100% |████████████████████████████████| 1.4MB 18.8MB/s 
menpo 0.8.1 has requirement matplotlib<2.0,>=1.4, but you'll have matplotlib 3.0.2 which is incompatible.
menpo 0.8.1 has requirement pillow<5.0,>=3.0, but you'll have pillow 5.4.0 which is incompatible.
menpo 0.8.1 has requirement scipy<1.0,>=0.16, but you'll have scipy 1.2.0 which is incompatible.
fastai 1.0.42 has requirement torch>=1.0.0, but you'll have torch 0.4.1 which is incompatible.
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1
     |████████████████████████████████| 676.9MB 67kB/s 
  Found existing installation: torch 0.4.1
    Uninstalling torch-0.4.1:
      Successfully uninstalled torch-0.4.1
     |████████████████████████████████| 71kB 1.7MB/s 
     |████████████████████████████████| 1.4MB 5.0MB/s 
     |████████████████████████████████| 204kB 42.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/ee/87/ce/a5023722374ca73b57fc8d4284ba6f973c0121

Using TensorFlow backend.


In [3]:
class Logger:

    def __init__(self, model_name, data_name):
        
        self.model_name = model_name
        self.data_name = data_name

        self.comment = '{}_{}'.format(model_name, data_name)
        self.data_subdir = '{}/{}'.format(model_name, data_name)

        # TensorBoard
        self.writer = SummaryWriter(comment=self.comment)

    def log(self, d_error, g_error, nrmse, epoch, n_batch, num_batches):

        if isinstance(d_error, torch.autograd.Variable):
            d_error = d_error.data.cpu().numpy()
        if isinstance(g_error, torch.autograd.Variable):
            g_error = g_error.data.cpu().numpy()

        step = Logger._step(epoch, n_batch, num_batches)
        self.writer.add_scalar(self.comment + "/D Error", d_error, step)
        self.writer.add_scalar(self.comment + "/D(G(x)) Error", g_error[0] + g_error[1], step)
        self.writer.add_scalar(self.comment + "/Dice", nrmse, step)
        
    def add_images(self, images, nrows, direc, step, normalize):
        
        img_name = '{}/images{}'.format(self.comment, str(direc))

        horizontal_grid = vutils.make_grid(
            images, normalize=normalize, scale_each=True)

        self.writer.add_image(img_name, horizontal_grid, step)
        
    def log_images(self, generated, real, sampled, num_images, epoch, n_batch, num_batches, format='NCHW', normalize=True):
        if type(generated) == np.ndarray:
            generated = torch.from_numpy(generated)
            
        if type(real) == np.ndarray:
            real = torch.from_numpy(real)
            
        if type(sampled) == np.ndarray:
            sampled = torch.from_numpy(sampled)
            
        if format=='NHWC':
            generated = generated.transpose(1,3)
            real = real.transpose(1,3)
            sampled = sampled.transpose(1, 3)

        step = Logger._step(epoch, n_batch, num_batches)
        nrows = int(np.sqrt(num_images))

        self.add_images(generated, nrows, '/generated_mask', step, normalize)
        self.add_images(sampled, nrows, '/original_mri', step, normalize)
        self.add_images(real, nrows, '/ground_truth_mask', step, normalize)
        

    def save_torch_images(self, horizontal_grid, grid, epoch, n_batch, plot_horizontal=True):
        out_dir = './data/images/{}'.format(self.data_subdir)
        Logger._make_dir(out_dir)

        # Plot and save horizontal
        fig = plt.figure(figsize=(128, 128))
        plt.imshow(np.moveaxis(horizontal_grid.numpy(), 0, -1))
        plt.axis('off')
        if plot_horizontal:
            display.display(plt.gcf())
        self._save_images(fig, epoch, n_batch, 'hori')
        plt.close()

        # Save squared
        fig = plt.figure()
        plt.imshow(np.moveaxis(grid.numpy(), 0, -1))
        plt.axis('off')
        self._save_images(fig, epoch, n_batch)
        plt.close()

    def _save_images(self, fig, epoch, n_batch, comment=''):
        out_dir = './data/images/{}'.format(self.data_subdir)
        Logger._make_dir(out_dir)
        fig.savefig('{}/{}_epoch_{}_batch_{}.png'.format(out_dir,
                                                         comment, epoch, n_batch))

    def display_status(self, epoch, num_epochs, n_batch, num_batches, d_error, g_error, d_pred_real, d_pred_fake, 
                       d_acc_real, d_acc_fake):
        
        # var_class = torch.autograd.variable.Variable
        if isinstance(d_error, torch.autograd.Variable):
            d_error = d_error.data.cpu().numpy()
        if isinstance(g_error, torch.autograd.Variable):
            g_error = g_error.data.cpu().numpy()
        if isinstance(d_pred_real, torch.autograd.Variable):
            d_pred_real = d_pred_real.data
        if isinstance(d_pred_fake, torch.autograd.Variable):
            d_pred_fake = d_pred_fake.data
        
        clear_output()
        print('Epoch: [{}/{}], Batch Num: [{}/{}]'.format(
            epoch,num_epochs, n_batch, num_batches)
             )
        print('Discriminator Loss: {:.4f}'.format(d_error))
        print('    Generator Loss: {:.4f}'.format(g_error[0] + g_error[1]))
        #print('Discriminator Loss: {:.4f}, Generator Loss: {:.4f}'.format(d_error, g_error))
        print('D(x) Loss: {:.4f}, D(G(x)) Loss: {:.4f}'.format(d_pred_real.mean(), d_pred_fake.mean()))
        print('D(x) Acc:  {:.4f}, D(G(x)) Acc:  {:.4f}'.format(d_acc_real, d_acc_fake))

    def save_models(self, generator, discriminator, epoch):
        out_dir = './data/models/{}'.format(self.data_subdir)
        Logger._make_dir(out_dir)
        torch.save(generator.state_dict(),
                   '{}/G_epoch_{}'.format(out_dir, epoch))
        torch.save(discriminator.state_dict(),
                   '{}/D_epoch_{}'.format(out_dir, epoch))

    def close(self):
        self.writer.close()

    # Private Functionality

    @staticmethod
    def _step(epoch, n_batch, num_batches):
        return epoch * num_batches + n_batch

    @staticmethod
    def _make_dir(directory):
        try:
            os.makedirs(directory)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise
                

In [4]:
import warnings;
warnings.filterwarnings('ignore');

class MSDataset(Dataset): 
          
    def __init__(self, transforms=None):
        
        self.transforms = transforms
        
        direc = "/floyd/home"
        
        os.chdir(direc)
        user = getpass('BitBucket user')
        password = getpass('BitBucket password')
        os.environ['BITBUCKET_AUTH'] = user + ':' + password.replace("@", "%40")

        !git clone https://$BITBUCKET_AUTH@bitbucket.org/sidijju/msdata.git
        
        root = direc + "/msdata/"
        
        self.mri = []
        self.mask = []
        theta = (np.arange(-15, 15, 15) * np.pi) / 180.
        
        good_indices = []
        
        print("Started file processing")
        
        index = 0
        for filename in sorted(os.listdir(root)):
            print(filename)
            if "FLAIR" in filename:
                img = nib.load(root + filename)
                dat = img.get_data()
                dat = np.rollaxis(dat, 1)
                for image in dat:
                    image = image.astype(float)
                    image = np.expand_dims(preprocessing.normalize(image), axis = -1)
                    image = image[:, 150:480, :]
                    self.mri.append(image)
                    
            if "mask" in filename:
                img = nib.load(root + filename)
                dat = img.get_data()
                dat = np.rollaxis(dat, 1)
                for image in dat:
                    image = image.astype(float)
                    image = np.expand_dims(image, axis = -1)
                    image = image[:, 150:480, :]
                    if not (np.amax(image) == 0.0):
                        good_indices.append(index)
                    self.mask.append(image)
                    index += 1
        
        print("Completed file processing")
        
        self.mri_final = []
        self.mask_final = []
        
        for j in range(1):
            for i in good_indices:
                self.mri_final.append(self.mri[i])
                self.mask_final.append(self.mask[i])

        self.mri = np.dstack(self.mri_final)
        self.mask = np.dstack(self.mask_final)
        
        self.data = np.stack((self.mri, self.mask), axis=3)
        
        self.data = np.moveaxis(self.data, 0, 2)
        self.data = np.moveaxis(self.data, 0, 1)
        
        print("Completed")
        
        print(self.data.shape)

    def __getitem__(self, index):
        return self.data[index]
        
    def __len__(self):
        return len(self.data)

In [5]:
r_dim = 330
c_dim = 192

image_shape = (r_dim, c_dim, 1)

dataset = MSDataset()

BitBucket user 
BitBucket password 


fatal: destination path 'msdata' already exists and is not an empty directory.
Started file processing
.git
patient26_FLAIR.nii.gz
patient26_mask.nii.gz
patient27_FLAIR.nii.gz
patient27_mask.nii.gz
patient28_FLAIR.nii.gz
patient28_mask.nii.gz
patient29_FLAIR.nii.gz
patient29_mask.nii.gz
patient30_FLAIR.nii.gz
patient30_mask.nii.gz
Completed file processing
Completed
(1021, 330, 192, 2)


## Define Different Types of Blocks

---
We will use two types of blocks in this GAN. 

### Resnet Block


---
The Resnet block consists of two convolution and activation layers, along with a direct connection between the input and the output layer. This serves to allow more information from the original image to flow through in the neural network.

![Diagram of Resnet Block](https://cdn-images-1.medium.com/max/1200/1*ByrVJspW-TefwlH7OLxNkg.png)

### Unet


---

The Unet consists of two different segments, the encoder and decoder segments. The difference between a standard encoder decoder and a Unet is that Unet also employs residual connections similar to the Resnet block. While generally used for segmentation based tasks, the Unet will be used in this task to determine areas of the image that are incorrect and thus de-alias the image. 

![Diagram of Unet](https://gluon.mxnet.io/_images/Pixel2pixel-Unet.png)

In [6]:
def convolution_block(x, filters, kernel_size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, kernel_size=kernel_size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    if activation == True:
        x = Activation("relu")(x)
    return x

def res_block(input, filters, kernel_size = (3,3), strides=(1,1), use_dropout=False):
    x = Activation("relu")(input)
    x = BatchNormalization()(x)
    x = convolution_block(x, filters, kernel_size )
    x = convolution_block(x, filters, kernel_size, activation=False)
    x = Add()([x, input])
    return x
  

## Define the Generator and the Discriminator

---

### Generator


---

The Generator model will take in an input of an aliased MRI image and output a de-aliased one. Its goal is to increase its chances of fooling the discriminator into thinking that it's producing the actual real image. 


In [7]:
#Number of filters
ngf = 64

#Input and Output number of channels
input_nc = 1
output_nc = 1

#Input Shape of the Generator
input_shape_generator = (330, 192, input_nc)

def generator_model():
    
    concat_axis = -1

    inputs = Input(shape=input_shape_generator)
                          
    conv1 = Conv2D(64, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(96, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(96, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)

    up_conv5 = UpSampling2D(size=(2, 2))(conv5)
    ch, cw = get_crop_shape(conv4, up_conv5)
    crop_conv4 = Cropping2D(cropping=(ch,cw))(conv4)
    up6 = concatenate([up_conv5, crop_conv4], axis=concat_axis)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up_conv6 = UpSampling2D(size=(2, 2))(conv6)
    ch, cw = get_crop_shape(conv3, up_conv6)
    crop_conv3 = Cropping2D(cropping=(ch,cw))(conv3)
    up7 = concatenate([up_conv6, crop_conv3], axis=concat_axis) 
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up_conv7 = UpSampling2D(size=(2, 2))(conv7)
    ch, cw = get_crop_shape(conv2, up_conv7)
    crop_conv2 = Cropping2D(cropping=(ch,cw))(conv2)
    up8 = concatenate([up_conv7, crop_conv2], axis=concat_axis)
    conv8 = Conv2D(96, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(96, (3, 3), activation='relu', padding='same')(conv8)

    up_conv8 = UpSampling2D(size=(2, 2))(conv8)
    ch, cw = get_crop_shape(conv1, up_conv8)
    crop_conv1 = Cropping2D(cropping=(ch,cw))(conv1)
    up9 = concatenate([up_conv8, crop_conv1], axis=concat_axis)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)

    ch, cw = get_crop_shape(inputs, conv9)
    conv9 = ZeroPadding2D(padding=((ch[0], ch[1]), (cw[0], cw[1])))(conv9)
    conv10 = Conv2D(1, (1, 1), activation = 'softmax')(conv9)

    model = Model(input = inputs, output = conv10)

    return model

def get_crop_shape(target, refer):
        # width, the 3rd dimension
        cw = (target.get_shape()[2] - refer.get_shape()[2]).value
        assert (cw >= 0)
        if cw % 2 != 0:
            cw1, cw2 = int(cw/2), int(cw/2) + 1
        else:
            cw1, cw2 = int(cw/2), int(cw/2)
        # height, the 2nd dimension
        ch = (target.get_shape()[1] - refer.get_shape()[1]).value
        assert (ch >= 0)
        if ch % 2 != 0:
            ch1, ch2 = int(ch/2), int(ch/2) + 1
        else:
            ch1, ch2 = int(ch/2), int(ch/2)

        return (ch1, ch2), (cw1, cw2)

### Discriminator


---

The Discriminator determines whether the output of the Generator is a real MRI image or whether it is "fake". Its goal is to be able to tell apart generated images from real ones.

In [8]:
#Number of filters
ndf = 64

#Number of output channels
output_nc = 1

#Input Shape of Image to the Discriminator
input_shape_discriminator = (330, 192, output_nc)


def discriminator_model():
    n_layers, use_sigmoid = 3, True
    inputs = Input(shape=input_shape_discriminator)

    x = Conv2D(filters=ndf, kernel_size=(4,4), strides=2, padding='same')(inputs)
    x = LeakyReLU(0.2)(x)
    x = Dropout(0.5)(x)

    nf_mult, nf_mult_prev = 1, 1
    for n in range(n_layers):
        nf_mult_prev, nf_mult = nf_mult, min(2**n, 8)
        x = Conv2D(filters=ndf*nf_mult, kernel_size=(4,4), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)
        x = Dropout(0.5)(x)

    nf_mult_prev, nf_mult = nf_mult, min(2**n_layers, 8)
    x = Conv2D(filters=ndf*nf_mult, kernel_size=(4,4), strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    x = Dropout(0.5)(x)

    x = Conv2D(filters=1, kernel_size=(4,4), strides=1, padding='same')(x)
    if use_sigmoid:
        x = Activation('sigmoid')(x)

    x = Flatten()(x)
    x = Dense(1024, activation='tanh')(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=x, name='Discriminator')
    return model

## Putting It all Together


---

In this segment we combine the generator and discriminator models and define losses for each of them. Since this model has to have the generated images be as close to the real ones as possible, we have to have the outputs of the generated images in the final model.

In [9]:
def gendis_mult_out(generator, discriminator):
    inputs = Input(shape=(330, 192, 1))
    generated_images = generator(inputs)
    outputs = discriminator(generated_images)
    model = Model(inputs=inputs, outputs=[generated_images, outputs])
    return model

### Losses


---

We will use multiple losses in the model. 
#### Wasserstein Loss


---


Wasserstein Loss is similar to Mean Squared Error in that it finds the mean of the differences between two images. It is known to improve GAN performance.

In [10]:
def w_loss(y_true, y_pred):
    return K.mean(y_true - y_pred)

#### Perceptual Loss


---

Perceptual Loss uses the classic VGG16 architecture to determine similarity between images.

In [11]:
from keras.applications.vgg16 import VGG16

def perceptual_loss(y_true, y_pred):
    vgg = VGG16(include_top=False, weights='imagenet', input_shape=(g_res, g_res, 3))
    loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
    loss_model.trainable = False
    y_true = K.concatenate((y_true, y_true, y_true), axis = 3)
    y_pred = K.concatenate((y_pred, y_pred, y_pred), axis = 3)
    loss_model_true = loss_model(y_true)
    loss_model_pred = loss_model(y_pred)
    mean = K.mean(K.square(loss_model_true - loss_model_pred))
    return K.mean(K.square(loss_model(y_true) - loss_model(y_pred)))

#### Mean Squared Error

___

MSE calculates the pixel by pixel loss from the true output to the fake one. 

In [12]:
def mse(y_true, y_pred):
    total = np.sum(np.square(np.subtract(y_true, y_pred)))/(g_res ** 2)
    #print("mse: ", total)
    return total
                

In [13]:
def fft_mse(y_true, y_pred):
    y_true_fft = tf.spectral.fft3d(tf.cast(y_true, tf.complex64))
    y_pred_fft = tf.spectral.fft3d(tf.cast(y_pred, tf.complex64))
    total = mse(tf.cast(y_true_fft, tf.float32), 
                tf.cast(y_pred_fft, tf.float32))
    return total

In [35]:
smooth = 1.
def dice(y_true, y_pred):
    positives = K.greater_equal(y_pred, 0.5)
    positives = K.cast(positives, K.floatx())
    outputs = positives + ((1-positives)*y_pred)
    y_true_f = K.abs(K.flatten(y_true))
    y_pred_f = K.abs(K.flatten(outputs))
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1.-dice(y_true, y_pred)

In [36]:
def g_loss(y_true, y_pred):
    dice_loss = dice_coef_loss(y_true, y_pred)
    return dice_loss


## Training the Model


---

We have now arrived at the final step of the GAN: defining the dataset and training the model. 

### Dataset


---

We will load in the OASIS Dataset using the custom class defined above. Since OASIS data is not public, I used a Bitbucket login to access the data and pull it into this notebook. This notebook will not work without that data. 

We will also define a method of removing information from the images in order to get aliased MRI images. This data will be x_train. 

The real images themselves will be y_train.

### Models and Optimizers


---

Here we will initialize the Generator and Discriminator models and also define the optimizers for each. We will also define functions for metrics that the model will keep track of, but NOT train on over time (SSIM, NMSE, PSNR). 

#### PSNR

PSNR or Peak Signal-to-Noise Ratio is a measure of the noise introduced by error between two given images. The closer PSNR is to 100, the better quality the reconstruction is.

In [37]:
import math
def psnr(y_true, y_pred):
    mse_val = mse(y_true, y_pred)
    if mse_val == 0:
        return 100
    return math.log10(255.0/mse_val)

#### NRMSE

NRMSE or Normalized Root MSE is a version of MSE that takes into account the images it compares. The closer NRMSE is to 0, the better.

In [38]:
from skimage.measure import compare_nrmse as nrmse

#### SSIM

In [39]:
from skimage.measure import compare_ssim as ssim

### Compiling and Training

---

In this last step, we will compile our models with the losses we defined and train them. 

Now we will process and define the batches of images.

In [56]:
currEpoch = 0

EPOCH_NUM = 10

BATCH_SIZE = 2

In [52]:
from keras.models import load_model

def loss_func(y_true, y_pred):
    return dice_coef_loss(y_true, y_pred)

g = load_model('best_7919.h5', custom_objects={'loss_func': loss_func,'dice': dice, 
                                                       'dice_coef_loss': dice_coef_loss})
d = discriminator_model()
d_on_g = gendis_mult_out(g, d)

In [53]:
def save_all_weights(d, g, epoch_number, current_loss):
    now = datetime.datetime.now()
    os.chdir("/floyd/home/")
    save_dir = os.path.join("weights/", '{}_{}_{} run'.format(now.month, now.day, now.year))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    g.save_weights(os.path.join(save_dir, 'generator_{}.h5'.format(epoch_number + currEpoch)), True)
    d.save_weights(os.path.join(save_dir, 'discriminator_{}.h5'.format(epoch_number + currEpoch)), True)

In [54]:
# Initialize optimizers
g_opt = Adam(epsilon=1E-4)
d_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
d_on_g_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

# Compile models
d.trainable = True
d.compile(optimizer=d_opt, loss=w_loss)
d.trainable = False

loss = [g_loss, w_loss]
loss_weights = [100, 1]
d_on_g.compile(optimizer=d_on_g_opt, loss=loss, loss_weights=loss_weights)
d.trainable = True

In [59]:
output_true_batch, output_false_batch = np.random.uniform(low = 0.9, high = 1.0, size = (BATCH_SIZE, 1)), np.zeros((BATCH_SIZE, 1))

logger = Logger(model_name='MSSeGAN_v1', data_name='')

In [60]:
x_train, x_test, y_train, y_test = train_test_split(dataset.data[:, :, :, 0], dataset.data[:, :, :, 1], test_size=0.20)

x_train = np.expand_dims(x_train, axis = -1)
y_train = np.expand_dims(y_train, axis = -1)
x_test = np.expand_dims(x_test, axis = -1)
y_test = np.expand_dims(y_test, axis = -1)

In [ ]:
for epoch in tqdm.tqdm(range(EPOCH_NUM)):
    
    num_samples = int(x_train.shape[0])
    
    permutated_indexes = np.random.permutation(num_samples)
    num_batches = int(num_samples / BATCH_SIZE)
    
    d_losses = []
    d_on_g_losses = []
    
    for n_batch in range(num_batches):
        batch_indexes = permutated_indexes[n_batch*BATCH_SIZE:(n_batch+1)*BATCH_SIZE]

        sampled_batch = x_train[batch_indexes, :, :, :]
        full_batch = y_train[batch_indexes, :, :, :]

        generated_images = g.predict(x=sampled_batch, batch_size=BATCH_SIZE)

        d_loss_real = d.train_on_batch(full_batch, output_true_batch)
        d_loss_fake = d.train_on_batch(generated_images, output_false_batch)
        d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
        d_losses.append(d_loss)

        d.trainable = False

        d_on_g_loss = d_on_g.train_on_batch(sampled_batch, [full_batch, output_true_batch])
        d_on_g_losses.append(d_on_g_loss)

        d.trainable = True
        
        dice_val = 0
        for i in range(BATCH_SIZE):
            dice_val += K.eval(dice(K.cast(full_batch[i][:, :, 0], 'float32'), generated_images[i][:, :, 0]))

        dice_val /= BATCH_SIZE
        
        logger.log(d_loss, d_on_g_loss, dice_val, epoch, n_batch, num_batches) 
        logger.log_images(generated_images[0:3], full_batch[0:3], sampled_batch[0:3], 3, epoch, n_batch, num_batches, format='NHWC')   
    
    with open('log.txt', 'a+') as f:
        f.write('{} - {} - {}\n'.format(epoch, np.mean(d_losses), np.mean(d_on_g_losses)))

    save_all_weights(d, g, epoch, int(np.mean(d_on_g_losses)))







  0%|          | 0/10 [00:00<?, ?it/s]